In [65]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import math

In [2]:
df = pd.read_csv('../data/in/acrylic-csv/Report-Mapping-building-M-English.csv', delimiter=";")
df

,Point,Latitude,Logitude,Altitude,Accuracy,Source,X,Y,LocationId,LocationName,...,SSIDConnectedTo,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42
0,1,56.853932,14.832514,0,0,Map,63,462,1,Ground Floor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,56.853932,14.832514,0,0,Map,63,462,1,Ground Floor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,56.853932,14.832514,0,0,Map,63,462,1,Ground Floor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,56.853932,14.832514,0,0,Map,63,462,1,Ground Floor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,56.853932,14.832514,0,0,Map,63,462,1,Ground Floor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13057,257,56.853921,14.833024,0,0,Map,258,470,1,Ground Floor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13058,257,56.853921,14.833024,0,0,Map,258,470,1,Ground Floor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13059,257,56.853921,14.833024,0,0,Map,258,470,1,Ground Floor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13060,257,56.853921,14.833024,0,0,Map,258,470,1,Ground Floor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['Point', 'Latitude', 'Logitude', 'Altitude', 'Accuracy', 'Source', 'X',
       'Y', 'LocationId', 'LocationName', 'ScanId', 'ScanName', 'MacAddress',
       'InventoryName', 'SSID', 'Vendor', 'RSSI', 'SNR', 'Chan', 'Frequency',
       'ChanWidth', '802.11', 'Rates', 'WEP', 'WPA', 'WPA2', 'WPS', 'Type',
       'PacketsCount', 'PacketsFail', 'LatencyMs', 'BitsPerSec', 'ConnectedAP',
       'SSIDConnectedTo', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42'],
      dtype='object')

In [71]:
df2["RSSI"]

0       -79
1       -73
2       -75
3       -85
4       -84
         ..
13057   -81
13058   -64
13059   -78
13060   -85
13061   -79
Name: RSSI, Length: 13062, dtype: int64

In [24]:
df2 = df.drop(columns=['Accuracy', 'Altitude', 'SSIDConnectedTo'])

In [5]:
def filterAPsSSID(ssid):
    subset = df[df["SSID"] == ssid]
    aps = subset[["SSID", "MAC", "FirstSeen", "Channel", "CurrentLatitude", "CurrentLongitude"]]
    return aps.drop_duplicates()

In [49]:
def addMarker(data, map):
    popuptext = "SSID: {} <br> MAC: {} <br> Channel: {}".format(data["SSID"], data["MacAddress"], data["Chan"])
    #popuptext = "{}".format(data["MacAddress"])
    folium.Marker(location=[data["meanLat"], data["meanLon"]], icon=folium.Icon(icon="wifi", prefix="fa"), popup=popuptext).add_to(map)

In [22]:
def generateMapForSSID(df):
    map = folium.Map(location=[56.853997, 14.832758], zoom_start=19)
    df.apply(addMarker, args=[map], axis=1)
    return map

In [23]:
def getMacMean(mac, locationDF):
    rowsWithMac = locationDF[locationDF["MacAddress"] == mac]
    latMean = rowsWithMac["Latitude"].mean()
    lonMean = rowsWithMac["Logitude"].mean()
    return [latMean, lonMean]

In [25]:
uniqueMacVendors = df2[["MacAddress", "Vendor"]].drop_duplicates()

In [ ]:
# @TODO get mean "RSSI"

In [74]:
ssidDetails = df2[["MacAddress", "SSID", "Chan"]].drop_duplicates()
ssidDetails

,MacAddress,SSID,Chan
0,14:91:82:37:71:14,It-verkstad 2.4 Ghz,1
1,78:BC:1A:A1:27:40,eduroam,11
2,78:BC:1A:A1:27:42,LNU-Guest,11
3,78:BC:1A:A1:28:C2,LNU-Guest,11
4,AC:1F:09:0C:B0:91,RAK7268CV2_B091,7
...,...,...,...
10559,62:ED:00:87:96:4B,NaN,4
10566,8C:3B:AD:EF:73:B1,NETGEAR89,13
10568,B0:6A:41:D7:D9:CF,Boden,6
10878,90:9A:4A:87:A8:32,Halldén,3+7


In [39]:
uniqueMacVendors[['meanLat', 'meanLon']] = uniqueMacVendors.apply(lambda row: pd.Series(getMacMean(row["MacAddress"], df2)), axis=1)

In [47]:
uniqueMacVendors.shape

(157, 4)

In [38]:
uniqueMacWithWiFi = uniqueMacVendors.merge(ssidDetails, on="MacAddress", how="left")
uniqueMacWithWiFi

,MacAddress,Vendor,meanLat,meanLon,SSID,Chan
0,14:91:82:37:71:14,Belkin International Inc.,56.854067,14.832879,It-verkstad 2.4 Ghz,1
1,78:BC:1A:A1:27:40,Cisco Systems. Inc,56.854042,14.832672,eduroam,11
2,78:BC:1A:A1:27:42,Cisco Systems. Inc,56.854081,14.832811,LNU-Guest,11
3,78:BC:1A:A1:28:C2,Cisco Systems. Inc,56.853998,14.832772,LNU-Guest,11
4,AC:1F:09:0C:B0:91,shenzhen RAKwireless technology Co.Ltd,56.854009,14.832652,RAK7268CV2_B091,7
...,...,...,...,...,...,...
152,62:ED:00:87:96:4B,NaN,56.853870,14.833480,NaN,4
153,8C:3B:AD:EF:73:B1,NETGEAR,56.853870,14.833480,NETGEAR89,13
154,B0:6A:41:D7:D9:CF,Google. Inc.,56.853870,14.833480,Boden,6
155,90:9A:4A:87:A8:32,TP-LINK TECHNOLOGIES CO.LTD.,56.853875,14.833423,Halldén,3+7


In [50]:
generateMapForSSID(uniqueMacWithWiFi)